In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, LeakyReLU, Flatten, Conv2D
from tensorflow.keras.models import Sequential
import numpy as np

# -------------------------
# 1) BUILD GENERATOR (CNN)
# -------------------------
def build_generator():
    model = Sequential([
        Dense(7*7*256, activation="relu", input_dim=100),
        Reshape((7, 7, 256)),
        Conv2DTranspose(128, (4,4), strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(64, (4,4), strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(1, (7,7), activation="tanh", padding="same")
    ])
    return model

# -------------------------
# 2) BUILD DISCRIMINATOR
# -------------------------
def build_discriminator():
    model = Sequential([
        Conv2D(64, (3,3), strides=2, padding="same", input_shape=(28,28,1)),
        LeakyReLU(0.2),
        Flatten(),
        Dense(1, activation="sigmoid")
    ])
    return model

# -------------------------
# 3) TRAINING LOOP (SHORT)
# -------------------------
(X_train, _), _ = tf.keras.datasets.mnist.load_data()
X_train = (X_train.astype("float32") - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer="adam", loss="binary_crossentropy")

gan = Sequential([generator, discriminator])
discriminator.trainable = False
gan.compile(optimizer="adam", loss="binary_crossentropy")

# Train for a few steps for demo
for step in range(2000):
    # train discriminator
    idx = np.random.randint(0, X_train.shape[0], 32)
    real = X_train[idx]
    noise = np.random.randn(32, 100)
    fake = generator.predict(noise, verbose=0)

    d_loss_real = discriminator.train_on_batch(real, np.ones((32,1)))
    d_loss_fake = discriminator.train_on_batch(fake, np.zeros((32,1)))

    # train GAN (generator wants discriminator to output 1)
    noise = np.random.randn(32, 100)
    gan.train_on_batch(noise, np.ones((32,1)))

# Generate one sample
noise = np.random.randn(1, 100)
sample = generator.predict(noise, verbose=0)[0]
print("Generated CNN-MNIST sample shape:", sample.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
 

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Sequential
import numpy as np

# -------------------------
# 1) PREPARE DATA
# -------------------------
(X_train, _), _ = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0  # normalize
X_train = X_train.reshape(-1, 28, 28)  # sequence: 28 rows, 28 features

# -------------------------
# 2) BUILD RNN GENERATOR
# -------------------------
def build_rnn_generator():
    model = Sequential([
        Dense(128, activation="relu", input_dim=100),
        RepeatVector(28),   # output sequence length = 28 rows
        LSTM(128, return_sequences=True),
        TimeDistributed(Dense(28, activation="sigmoid"))   # each row = 28 pixels
    ])
    return model

# -------------------------
# 3) TRAIN SHORT
# -------------------------
gen = build_rnn_generator()
gen.compile(optimizer="adam", loss="mse")

# Tiny training loop (demo)
noise = np.random.randn(1000, 100)
gen.fit(noise, X_train[:1000], epochs=5, batch_size=32)

# Generate an MNIST-like image
sample = gen.predict(np.random.randn(1, 100), verbose=0)[0]
print("Generated RNN-MNIST sample shape:", sample.shape)


In [ ]:
import tensorflow as tf
import numpy as np

# ============================
# 1. Load MNIST
# ============================
(x_train, _), _ = tf.keras.datasets.mnist.load_data()

# Normalize pixel values from [0,255] → [0,1]
x_train = (x_train.astype(np.float32) / 255.0)

# Flatten each image (28x28 → 784 vector)
x_train = x_train.reshape(-1, 784)

# Input/output dimension = number of pixels
D = 784

# Hidden layer width (typical MADE uses large layers)
H = 512


# ============================
# 2. Mask creation functions
# ============================

def create_mask(input_size, hidden_sizes, output_size):
    """
    Creates the MADE masks for:
    - input → hidden1
    - hidden1 → hidden2
    - hidden_last → output

    The masks enforce autoregressive ordering.
    """

    degrees = []  # stores assigned "indices" for each layer

    # Degrees for input layer = 1..784
    # (Pixel 1 must come before pixel 2, etc.)
    degrees.append(np.arange(1, input_size + 1))

    # Degrees for hidden layers (random 1..D-1)
    # Ensures different neurons learn different dependencies
    for h in hidden_sizes:
        degrees.append(np.random.randint(1, input_size, size=h))

    # Degrees for output layer = 1..784 (same order as input)
    degrees.append(np.arange(1, output_size + 1))

    masks = []

    # Build masks for each layer:
    # Mask[l][i,j] = 1 if degree[i] ≤ degree[j]
    # i = neuron in previous layer
    # j = neuron in next layer
    for l in range(len(hidden_sizes) + 1):
        mask = (degrees[l][:, None] <= degrees[l + 1][None, :]).astype(np.float32)
        masks.append(mask)

    return masks, degrees



# ============================
# 3. Masked Dense Layer
# ============================

class MaskedDense(tf.keras.layers.Layer):
    """
    A Dense layer where weights are multiplied by a binary mask.
    This forces autoregressive constraints.
    """

    def __init__(self, units, mask=None, activation=None):
        super().__init__()
        self.units = units
        self.mask = mask  # the binary mask matrix
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        # Weight matrix W: (input_dim x units)
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform'
        )

        # Bias vector
        self.b = self.add_weight(
            shape=(self.units,),
            initializer='zeros'
        )

    def call(self, inputs):
        # Apply mask: zero out forbidden connections
        masked_w = self.w * self.mask

        # Standard dense forward pass
        out = tf.matmul(inputs, masked_w) + self.b

        # Apply activation (ReLU/sigmoid)
        if self.activation:
            return self.activation(out)
        return out



# ============================
# 4. Build MADE Model
# ============================

def build_made(input_size, hidden_sizes):
    """
    Builds the MADE network:
    Input → MaskedDense → MaskedDense → Output(MaskedDense)
    """

    # Create masks for all layers
    masks, degrees = create_mask(input_size, hidden_sizes, input_size)

    inputs = tf.keras.Input(shape=(input_size,))
    x = inputs

    # Hidden layers with masks
    for h, m in zip(hidden_sizes, masks[:-1]):
        x = MaskedDense(h, mask=m, activation='relu')(x)

    # Output layer with sigmoid (pixel probabilities)
    outputs = MaskedDense(input_size, mask=masks[-1], activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)

    # Bernoulli likelihood for each pixel
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model



# ============================
# 5. Train MADE
# ============================

made = build_made(D, [H, H])  # 2 hidden layers, 512 each
made.summary()

# MADE learns to reconstruct the image pixel-by-pixel
# but with autoregressive masking (no cheating)
made.fit(
    x_train,     # input image
    x_train,     # target is same image
    batch_size=256,
    epochs=5
)



# ============================
# 6. Sampling from MADE
# ============================

def sample_from_made(model, D=784):
    """
    Generates a new MNIST-like image using autoregressive sampling:
    x1 first, then x2 using x1, then x3 using x1,x2, etc.
    """

    sample = np.zeros((1, D), dtype=np.float32)

    # Loop pixel-by-pixel (1..784)
    for i in range(D):

        # Predict all pixel probabilities given partially filled sample
        out = model.predict(sample, verbose=0)

        prob = out[0, i]  # probability of pixel i being 1

        # Sample pixel i from Bernoulli(prob)
        sample[0, i] = (np.random.rand() < prob)

    # Convert vector → 28x28 image
    return sample.reshape(28, 28)



# Generate one MNIST-like sample
generated = sample_from_made(made)

import matplotlib.pyplot as plt
plt.imshow(generated, cmap='gray')
plt.title("MADE Sample")
plt.axis('off')
plt.show()
